In [67]:
import os
import re
import spacy
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [75]:
def get_surah_names():
    surah_names = [] #surah names sorted
    URL = "https://surahquran.com/quran-search/quran.html"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    all_table = soup.find_all('table')[1]
    for idx, elm in enumerate(all_table.find_all("a")):
        surah_names.append(
            (idx + 1, elm.text)
        )
    return surah_names

In [77]:
book = pd.read_csv(
    f'{os.getcwd()}/data/french.csv',
    sep="|",
    header=None,
)
book.columns = ["sura", "ayat", "text"]
book['sura'] = book['sura'].astype(int)
book['ayat'] = book['ayat'].astype(int)

mapping_dictionary = {t[0]: t for t in get_surah_names()}
book['sura_name'] = book['sura'].map(mapping_dictionary).apply(lambda t: t[1])

def cleaning_text(text):
    text = re.sub(r'` ', "'", text)
    text = re.sub(r' +(\.)', r'\1', text)
    return text

book["text"] = book["text"].apply(lambda x: cleaning_text(x))
book['text'].to_csv(f'{os.getcwd()}/output.txt', index=False, sep="\t", header=None)

In [60]:
nlp = spacy.load('fr_core_news_sm')

with open(f'{os.getcwd()}/output.txt') as file:
    text = file.read().replace('\n', ' ')

def find_sentences():
    doc = nlp(text)
    sentences = list(doc.sents)
    return sentences

In [80]:
book.tail(10)

,sura,ayat,text,sura_name
6111,113,2,"contre le mal des êtres qu´Il a créés,",الفلق
6112,113,3,contre le mal de l´obscurité quand elle s´appr...,الفلق
6113,113,4,contre le mal de celles qui soufflent (les sor...,الفلق
6114,113,5,"et contre le mal de l´envieux quand il envie "".",الفلق
6115,114,1,"Dis : ""Je cherche protection auprès du Seigneu...",الناس
6116,114,2,"Le Souverain des hommes,",الناس
6117,114,3,"Dieu des hommes,",الناس
6118,114,4,"contre le mal du mauvais conseiller, furtif,",الناس
6119,114,5,"qui souffle le mal dans les poitrines des hommes,",الناس
6120,114,6,"qu´il (le conseiller) soit un djinn, ou un êtr...",الناس
